In [13]:
import os, numpy as np, pandas as pd

In [14]:
import matplotlib.pyplot as plt
from sklearn.svm import LinearSVC

In [15]:
data_root = './datasets/'

In [16]:
adhd_uw_dir = 'adhd_uw/adhd'

In [17]:
td_uw_dir = 'adhd_uw/td'

In [18]:
adhd_graphs = []
td_graphs = []

In [19]:
for file in os.listdir(os.path.join(data_root, adhd_uw_dir)):
    adhd_graph = np.loadtxt(os.path.join(data_root, adhd_uw_dir, file), dtype=np.float)
    adhd_graphs.append(adhd_graph)
    del adhd_graph

In [20]:
for file in os.listdir(os.path.join(data_root, td_uw_dir)):
    td_graph = np.loadtxt(os.path.join(data_root, td_uw_dir, file), dtype=np.float)
    td_graphs.append(td_graph)
    del td_graph

In [21]:
def get_specific_graph_list(data_dir, group):
    
    specific_graph_list = []
    
    for file in os.listdir(os.path.join('./datasets', data_dir, group)):
        graph = np.loadtxt(os.path.join('./datasets', data_dir, group, file), dtype=np.float)
        specific_graph_list.append(adhd_graph)
        del graph
    
    return specific_graph_list

In [22]:
len(adhd_graphs)

190

In [23]:
len(td_graphs)

330

In [24]:
adhd_summary_graph = sum(adhd_graphs)/len(adhd_graphs)

In [25]:
td_summary_graph = sum(td_graphs)/len(td_graphs)

In [26]:
tmp_adhd_graph = np.loadtxt('./datasets/DTI/fun90/R_0.8/r003.txt', dtype=np.float)

In [27]:
tmp_adhd_graph.sum()

598.0

In [28]:
tmp_adhd_graph.shape

(90, 90)

In [29]:
598/90/90

0.07382716049382716

In [30]:
diff_net_adhd = adhd_summary_graph-td_summary_graph

In [31]:
diff_net_td = td_summary_graph-adhd_summary_graph

In [32]:
threshold = 0.001

In [33]:
mask_net_adhd = (diff_net_adhd>threshold).astype(np.int)

In [34]:
mask_net_adhd.sum()

17474

In [35]:
(diff_net_adhd!=0).astype(np.int).sum()

35796

In [36]:
tmp = diff_net_adhd[:5, :5]

In [37]:
tmp

array([[ 0.        , -0.00478469,  0.00956938, -0.00478469,  0.04082935],
       [-0.00478469,  0.        ,  0.03110048, -0.0185008 , -0.00318979],
       [ 0.00956938,  0.03110048,  0.        , -0.01068581, -0.00143541],
       [-0.00478469, -0.0185008 , -0.01068581,  0.        , -0.01100478],
       [ 0.04082935, -0.00318979, -0.00143541, -0.01100478,  0.        ]])

In [38]:
tmp[tmp==0] = 1

In [39]:
tmp

array([[ 1.        , -0.00478469,  0.00956938, -0.00478469,  0.04082935],
       [-0.00478469,  1.        ,  0.03110048, -0.0185008 , -0.00318979],
       [ 0.00956938,  0.03110048,  1.        , -0.01068581, -0.00143541],
       [-0.00478469, -0.0185008 , -0.01068581,  1.        , -0.01100478],
       [ 0.04082935, -0.00318979, -0.00143541, -0.01100478,  1.        ]])

In [40]:
tmp = np.zeros_like(tmp)

In [41]:
tmp

array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [42]:
tmp[diff_net_adhd[:5, :5]>threshold] = 1

In [43]:
tmp

array([[1., 0., 1., 0., 1.],
       [0., 1., 1., 0., 0.],
       [1., 1., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 1.]])

In [ ]:
np.power(np.linalg.norm(tmp), -10)

In [ ]:
import math

In [ ]:
math.pow(3, 0)

In [ ]:
def calculate_weights(template_graph, specific_graph, gamma=1):
    diff = np.linalg.norm(template_graph-specific_graph)
    return math.pow(diff, gamma/2-1)

In [ ]:
def calculate_weights(template_graph, specific_graph_list, gamma):
    weights = []
    
    for specific_graph in specific_graph_list:
        weight = update_weight(template_graph, specific_graph, gamma)
        weights.append(weight)
        del weight
    
    return weights/sum(weights)

In [ ]:
def get_summary_graph(specific_graph_list, weights):
    template_graph = np.zeros_like(specific_graph_list[0])
    
    for i in range(weights):
        template_graph += weights[i]*specific_graph_list[i]
        
    return template_graph

In [ ]:
def update_summary_graph(positive_summary_graph, negative_summary_graph, threshold=0.01, iterations = 10):
    positive_graph = positive_summary_graph
    negative_graph = negative_summary_graph
    
    # main loop
    for i in range(iterations):
        
        # updating the positive graph
        update_positive_graph = positive_graph[positive_graph>negative_graph+threshold]
        positive_graph = update_positive_graph
        
        # updating the positive graph
        update_negative_graph = negative_graph[negative_graph>positive_graph+threshold]
        negative_graph = update_negative_graph
    
    return positive_graph, negative_graph

In [ ]:
def get_summary_features(sample_graph, positive_graph, negative_graph):
    # edge nums
    positive_edges = sample_graph[positive_graph>0].sum()
    negative_edges = sample_graph[negative_graph>0].sum()
    
    return positive_edges, negative_edges

In [ ]:
def get_connnectivity_features(sample_graph, positive_graph, negative_graph):
    # connectivity feature with mask
    graph_index = positive_graph+negative_graph
    connectivity_matrix = sample_graph[graph_index>0]
    
    # upper triangle matrix 
    triu_mask = np.tril(np.ones_like(connectivity_matrix))==0
    
    return connectivity_matrix[triu_mask]

In [ ]:
np.triu(tmp, 1).reshape(-1)

In [ ]:
mask = np.tril(np.ones_like(tmp))

In [ ]:
tmp.shape

In [ ]:
tmp[mask==0]

In [ ]:
tmp

In [ ]:
diff_net_adhd.shape

In [ ]:
triu_mask = np.tril(np.ones_like(diff_net_adhd))==0

In [ ]:
diff_net_adhd[triu_mask].shape

In [ ]:
190*(190-1)/2

In [ ]:
np.concatenate([diff_net_adhd[triu_mask].reshape(1, -1), diff_net_adhd[triu_mask].reshape(1, -1)]).shape

In [ ]:
tmp = np.random.rand(5, 5)

In [ ]:
tmp

In [ ]:
threshod = np.percentile(tmp, 50)

In [ ]:
tmp[tmp<threshod] = 0

In [ ]:
tmp

In [ ]:
threshod

In [ ]:
def threshold_split_feature(input_file_dir, output_file_dir, ratio = 0.9):
    #  if not output_file_dir, generate the corresponding dir
    if not os.path.exists(output_file_dir):
        os.mkdir(output_file_dir)
        
    for txt_file in os.listdir(input_file_dir):
        # get the matrix from .txt file
        input_data = np.loadtxt(os.path.join(input_file_dir, txt_file), dtype=np.float)
        # set the threshold to split the feature
        threshold = np.percentile(input_data, ratio)
        # generate 0-1 connectivity matrix
        output_data = np.zeros_like(input_data)
        output_data[input_data>threshold] = 1
        np.savetxt(os.path.join(output_file_dir, txt_file, fmt="d", delimiter=" "))

TypeError: unsupported operand type(s) for /: 'list' and 'float'

In [4]:
pos_graph_list = get_specific_graph_list(data_dir, positive_group)

In [7]:
len(pos_graph_list)

49

In [8]:
neg_graph_list = get_specific_graph_list(data_dir, negative_group)

In [9]:
len(neg_graph_list)

52

In [5]:
weights = [1,1,1,1,1,1]

In [6]:
1/sum(weights)

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [7]:
neg_test_ids = list(range(neg_train_num) - set(neg_train_ids))

NameError: name 'neg_train_num' is not defined

In [1]:
import math

In [3]:
data = [0.20, 0.19, 0.18]

In [4]:
for i in range(len(data)):
    data[i] = math.exp(data[i])

In [5]:
data = [elem/sum(data) for elem in data]

In [6]:
data

[0.33667216652898474, 0.33332222249999355, 0.3300056109710218]

In [1]:
import numpy as np

In [3]:
np.arange(0, 2, 0.1)

array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9])

In [ ]:
0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9